<a href="https://colab.research.google.com/github/Tekleab15/Regularized_Auto_Encoder/blob/main/RAE_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularized Autoencoder Implementation
This notebook demonstrates the implementation of a Regularized Autoencoder (RAE) using backpropagation, inspired by the methodologies described in "The Neural Coding Framework for Learning Generative Models".



## Dataset Loading and Preprocessing:
All the four datasets used in the paper (MNIST, KMNIST, FMNIST, CalTech101)

# 1.Preprocessing the datasets

In [1]:
# Importing Required Libraries and Classes
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
# Normalize and convert to binary
def preprocess_dataset(x_train, x_test):
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = (x_train > 0.5).astype('float32')
    x_test = (x_test > 0.5).astype('float32')

    x_train = x_train.reshape((x_train.shape[0], 28, 28))
    x_test = x_test.reshape((x_test.shape[0], 28, 28))
    return x_train, x_test

In [3]:
# load mnist dataset as it's splitted into a training and testing sets
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


# 2. Loading KMNIST dataset

In [4]:
# Unlike MNIST datasets KMNIST datasets should be loaded in a little bit different way
(x_train_kmnist, y_test_kmnist), ds_info = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)
def convert_to_numpy(ds):
    images, labels = [], []
    for img, lbl in tfds.as_numpy(ds):
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

# Convert datasets to numpy arrays
x_train_kmnist, y_train_kmnist = convert_to_numpy(x_train_kmnist)
x_test_kmnist, y_test_kmnist = convert_to_numpy(y_test_kmnist)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.6D3OK7_3.0.1/kmnist-train.tfrecord*...:   0%|          |…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.6D3OK7_3.0.1/kmnist-test.tfrecord*...:   0%|          | …

Dataset kmnist downloaded and prepared to /root/tensorflow_datasets/kmnist/3.0.1. Subsequent calls will reuse this data.


# 3. Loading FMNIST dataset

In [5]:
# Loading the FMNIST dataset
(x_train_fmnist, y_test_fmnist), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

x_train_fmnist, y_train_fmnist = convert_to_numpy(x_train_fmnist)
x_test_fmnist, y_test_fmnist = convert_to_numpy(y_test_fmnist)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/incomplete.S5S3I9_3.0.1/fashion_mnist-train.tfrecord*...:   …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/incomplete.S5S3I9_3.0.1/fashion_mnist-test.tfrecord*...:   0…

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


# Preprocessing for MNIST, KMNIST, FMNIST datasets

In [6]:
# Preprocessing the MNIST dataset
x_train_mnist, x_test_mnist = preprocess_dataset(x_train_mnist, x_test_mnist)
# Preprocessing the KMNIST dataset
x_train_kmnist, x_test_kmnist = preprocess_dataset(x_train_kmnist, x_test_kmnist)
# Preprocessing the FMNIST dataset
x_train_fmnist, x_test_fmnist = preprocess_dataset(x_train_fmnist, x_test_fmnist)

print("Mnist train shape:", x_train_mnist.shape)
print("Mnist test shape:", x_test_mnist.shape)
print("Kmnist train shape:", x_train_kmnist.shape)
print("Kmnist test shape:", x_test_kmnist.shape)
print("Fmnist train shape:", x_train_fmnist.shape)
print("Fmnist test shape:", x_test_fmnist.shape)

Mnist train shape: (60000, 28, 28)
Mnist test shape: (10000, 28, 28)
Kmnist train shape: (60000, 28, 28)
Kmnist test shape: (10000, 28, 28)
Fmnist train shape: (60000, 28, 28)
Fmnist test shape: (10000, 28, 28)


# 4. Loading and Preprocessing CalTech101

In [ ]:
# Preprocess images: normalize, resize, and threshold to binary
def preprocess_image(image, label, image_size=(16, 16)):
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize
    image = tf.image.resize(image, image_size)  # Resize
    image = tf.cast(image > 0.5, tf.float32)  # Threshold to binary
    return image, label

# Load CalTech101 dataset
(ds_train, ds_test), ds_info = tfds.load(
    'caltech101',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Apply preprocessing to the dataset
ds_train = ds_train.map(lambda img, lbl: preprocess_image(img, lbl))
ds_test = ds_test.map(lambda img, lbl: preprocess_image(img, lbl))

# Convert to numpy arrays
def convert_to_numpy(ds):
    images, labels = [], []
    for img, lbl in tfds.as_numpy(ds):
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

x_train_caltech, y_train_caltech = convert_to_numpy(ds_train)
x_test_caltech, y_test_caltech = convert_to_numpy(ds_test)

# Flatten the images for the autoencoder input
x_train_caltech = x_train_caltech.reshape((x_train_caltech.shape[0], -1))
x_test_caltech = x_test_caltech.reshape((x_test_caltech.shape[0], -1))

print("CalTech101 train shape:", x_train_caltech.shape)
print("CalTech101 test shape:", x_test_caltech.shape)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
# Calculating Log-liklihood
def calculate_log_likelihood(autoencoder, data, num_samples=5000):
    log_likelihoods = []
    for _ in range(num_samples):
        reconstructed = autoencoder(data)
        log_likelihood = -tf.keras.losses.binary_crossentropy(data, reconstructed)
        log_likelihoods.append(log_likelihood)
    return np.mean(log_likelihoods)

In [ ]:
class RegularizedAutoencoder(tf.keras.Model):
    def __init__(self, input_shape):
        super(RegularizedAutoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=input_shape),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(20, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dense(np.prod(input_shape), activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Reshape(input_shape)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# MSE calculation
def masked_mse(y_true, y_pred, mask):
    return np.mean(((y_true - y_pred) * (1 - mask)) ** 2)

# MNIST training

In [18]:
# Instantiate and compile the model
mnist_input_shape = x_train_mnist.shape[1:]
mnist_autoencoder = RegularizedAutoencoder(mnist_input_shape)
mnist_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
history = mnist_autoencoder.fit(x_train_mnist, x_train_mnist, epochs=50, batch_size=200, validation_data=(x_test_mnist, x_test_mnist))

# Evaluate the model
mnist_bce = mnist_autoencoder.evaluate(x_test_mnist, x_test_mnist)
print(f'MNIST Binary Cross-Entropy (BCE): {mnist_bce}')

# Create a mask and calculate M-MSE
MNIST_mask = np.random.choice([0, 1], size=x_test_mnist.shape, p=[0.5, 0.5])
MNIST_mse = masked_mse(x_test_mnist, mnist_autoencoder.predict(x_test_mnist), MNIST_mask)
print(f'MNIST Masked Mean Squared Error (M-MSE): {MNIST_mse}')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 0.2922 - val_loss: 0.1360
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - loss: 0.1292 - val_loss: 0.1107
Epoch 3/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - loss: 0.1097 - val_loss: 0.1007
Epoch 4/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - loss: 0.1008 - val_loss: 0.0934
Epoch 5/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 0.0934 - val_loss: 0.0880
Epoch 6/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - loss: 0.0881 - val_loss: 0.0843
Epoch 7/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - loss: 0.0842 - val_loss: 0.0806
Epoch 8/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - loss: 0.0803 - val_loss: 0.0775
Epoch 9/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 0.0775 - val_loss: 0.0756
Epoch 10/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - loss: 0.0753 - val_loss: 0.0730
Epoch 11/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - loss: 0.0728 - val_loss: 0.0714
Epoch 12/50
300/300 ━━━━━━━━━━━━

In [ ]:
# Get the latent representations from the encoder
mnist_latent_train = mnist_autoencoder.encoder.predict(x_train_mnist)
mnist_latent_test = mnist_autoencoder.encoder.predict(x_test_mnist)

# Train a logistic regression classifier
classifier = LogisticRegression(max_iter=500)
classifier.fit(mnist_latent_train, y_train_mnist)

# Predict on the test set
y_pred = classifier.predict(mnist_latent_test)

# Calculate classification error
mnist_classification_error = 1 - accuracy_score(y_test_mnist, y_pred)
print(f'MNIST Classification Error: {mnist_classification_error}')

MNIST Classification Error claculation

In [ ]:
# Get the latent representations from the encoder
mnist_latent_train = mnist_autoencoder.encoder.predict(x_train_mnist)
mnist_latent_test =  mnist_autoencoder.encoder.predict(x_test_mnist)

# Scale the latent representations
scaler = StandardScaler()
mnist_latent_train_scaled = scaler.fit_transform(mnist_latent_train)
mnist_latent_test_scaled = scaler.transform(mnist_latent_test)

# Train a logistic regression classifier
mnist_classifier = LogisticRegression(max_iter=2000)
mnist_classifier.fit(mnist_latent_train_scaled, y_train_mnist)

# Predict on the test set
mnist_y_pred = classifier.predict(mnist_latent_test_scaled)

# Calculate classification error
mnist_classification_error = 1 - accuracy_score(y_test_mnist, y_pred)
print(f'Classification Error: {mnist_classification_error:.2%}')

# MNIST likelihood error
log_likelihood = calculate_log_likelihood(mnist_autoencoder, x_test_mnist)
print(f'MNIST Log-Likelihood: {log_likelihood:.2%}')

KMNIST **Training**

In [11]:
kmnist_input_shape = x_train_kmnist.shape[1:]
kmnist_autoencoder = RegularizedAutoencoder(kmnist_input_shape)

# Compile the model
kmnist_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
kmnist_history = kmnist_autoencoder.fit(x_train_kmnist, x_train_kmnist, epochs=50, batch_size=200, validation_data=(x_test_kmnist, x_test_kmnist))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 15s 32ms/step - loss: 0.4268 - val_loss: 0.2928
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - loss: 0.2681 - val_loss: 0.2622
Epoch 3/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - loss: 0.2418 - val_loss: 0.2509
Epoch 4/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.2309 - val_loss: 0.2450
Epoch 5/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 12s 34ms/step - loss: 0.2239 - val_loss: 0.2414
Epoch 6/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 0.2203 - val_loss: 0.2389
Epoch 7/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 0.2161 - val_loss: 0.2362
Epoch 8/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 0.2143 - val_loss: 0.2350
Epoch 9/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.2118 - val_loss: 0.2332
Epoch 10/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 0.2103 - val_loss: 0.2330
Epoch 11/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - loss: 0.2088 - val_loss: 0.2310
Epoch 12/50
300/300 ━━━━━━━━━━━━━━

In [17]:
# Evaluate BCE
kmnist_bce = kmnist_autoencoder.evaluate(x_test_kmnist, x_test_kmnist)
print(f'KMNIST Binary Cross-Entropy (BCE): {round(kmnist_bce, 3)}')

# Calculate M-MSE
def masked_mse(y_true, y_pred, mask):
    return np.mean(((y_true - y_pred) * (1 - mask)) ** 2)

kmnist_mask = np.random.choice([0, 1], size=x_test_kmnist.shape, p=[0.5, 0.5])
kmnist_mse = masked_mse(x_test_kmnist, kmnist_autoencoder.predict(x_test_kmnist), kmnist_mask)
print(f'KMNIST Masked Mean Squared Error (M-MSE): {round(kmnist_mse, 3)}')

#Classification Error

# Get the latent representations from the encoder
kmnist_latent_train = kmnist_autoencoder.encoder.predict(x_train_kmnist)
kmnist_latent_test =  kmnist_autoencoder.encoder.predict(x_test_kmnist)

# Scale the latent representations
kmnist_scaler = StandardScaler()
kmnist_latent_train_scaled = kmnist_scaler.fit_transform(kmnist_latent_train)
kmnist_latent_test_scaled =  kmnist_scaler.transform(kmnist_latent_test)

# Train a logistic regression classifier
kmnist_classifier = LogisticRegression(max_iter=2000)
kmnist_classifier.fit(kmnist_latent_train_scaled, y_train_kmnist)

# Predict on the test set
y_pred = kmnist_classifier.predict(kmnist_latent_test_scaled)

# Calculate classification error
kmnist_classification_error = 1 - accuracy_score(y_test_kmnist, y_pred)
print(f'KMNIST Classification Error: {round(kmnist_classification_error,3):.2%}')
# KMNIST likelihood error
log_likelihood = calculate_log_likelihood(kmnist_autoencoder, x_test_kmnist)
print(f'KMNIST Log-Likelihood: {log_likelihood:.2%}')


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2140
KMNIST Binary Cross-Entropy (BCE): 0.215
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
KMNIST Masked Mean Squared Error (M-MSE): 0.033
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
KMNIST Classification Error: 37.40%


Training Auto Encoder using FMNIST dataset

In [19]:
fmnist_input_shape = x_train_fmnist.shape[1:]
fmnist_autoencoder = RegularizedAutoencoder(fmnist_input_shape)

# Compile the model
fmnist_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
fmnist_history = fmnist_autoencoder.fit(x_train_fmnist, x_train_fmnist, epochs=50, batch_size=200, validation_data=(x_test_fmnist, x_test_fmnist))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 0.3569 - val_loss: 0.1860
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - loss: 0.1781 - val_loss: 0.1676
Epoch 3/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 0.1617 - val_loss: 0.1582
Epoch 4/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 0.1540 - val_loss: 0.1534
Epoch 5/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - loss: 0.1486 - val_loss: 0.1479
Epoch 6/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 0.1438 - val_loss: 0.1445
Epoch 7/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 0.1403 - val_loss: 0.1430
Epoch 8/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.1381 - val_loss: 0.1382
Epoch 9/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - loss: 0.1349 - val_loss: 0.1376
Epoch 10/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.1331 - val_loss: 0.1353
Epoch 11/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.1321 - val_loss: 0.1340
Epoch 12/50
300/300 ━━━━━━━━━━━━━━━━━

In [21]:
# Measuring performance of the Autoencoder trained with FMNIST datasets
# Evaluate BCE
fmnist_bce = fmnist_autoencoder.evaluate(x_test_fmnist, x_test_fmnist)
print(f'Binary Cross-Entropy (BCE): {fmnist_bce}')

# Calculate M-MSE
def masked_mse(y_true, y_pred, mask):
    return np.mean(((y_true - y_pred) * (1 - mask)) ** 2)

fmnist_mask = np.random.choice([0, 1], size=x_test_fmnist.shape, p=[0.5, 0.5])
fmnist_mse = masked_mse(x_test_fmnist, fmnist_autoencoder.predict(x_test_fmnist), fmnist_mask)
print(f'Masked Mean Squared Error (M-MSE): {fmnist_mse}')

# Get the latent representations from the encoder
fmnist_latent_train = fmnist_autoencoder.encoder.predict(x_train_fmnist)
fmnist_latent_test =  fmnist_autoencoder.encoder.predict(x_test_fmnist)

# Scale the latent representations
fmnist_scaler = StandardScaler()
fmnist_latent_train_scaled = fmnist_scaler.fit_transform(fmnist_latent_train)
fmnist_latent_test_scaled =  fmnist_scaler.transform(fmnist_latent_test)

# Train a logistic regression classifier
fmnist_classifier = LogisticRegression(max_iter=2000)
fmnist_classifier.fit(fmnist_latent_train_scaled, y_train_fmnist)

# Predict on the test set
y_pred = fmnist_classifier.predict(fmnist_latent_test_scaled)

# Calculate classification error
fmnist_classification_error = 1 - accuracy_score(y_test_fmnist, y_pred)
print(f'FMNIST Classification Error: {fmnist_classification_error:.2%}')
# FMNIST likelihood error
log_likelihood = calculate_log_likelihood(fmnist_autoencoder, x_test_fmnist)
print(f'FMNIST Log-Likelihood: {log_likelihood:.2%}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.1217
Binary Cross-Entropy (BCE): 0.12124451994895935
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Masked Mean Squared Error (M-MSE): 0.01781981034496288
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Classification Error: 23.89%


# Training Autoencoder Model using CalTech109

In [25]:
# Training Model using caltech109 dataset
caltech_input_shape = x_train_caltech.shape[1:]
caltech_autoencoder = RegularizedAutoencoder(caltech_input_shape)

# Compile the model
caltech_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
caltech_history = caltech_autoencoder.fit(x_train_caltech, x_train_caltech, epochs=50, batch_size=200, validation_data=(x_test_caltech, x_test_caltech))



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 0.6813 - val_loss: 0.6167
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.6100 - val_loss: 0.5702
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 0.5489 - val_loss: 0.5213
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.5077 - val_loss: 0.4937
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.4809 - val_loss: 0.4755
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.4668 - val_loss: 0.4643
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.4542 - val_loss: 0.4548
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.4452 - val_loss: 0.4480
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.4395 - val_loss: 0.4434
Epoch 10/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.4292 - val_loss: 0.4342
Epoch 11/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.4227 - val_loss: 0.4269
Epoch 12/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.4178 - val_l

In [ ]:
#Measuring the Performance of the Caltech model
# Evaluate BCE
caltech_bce = caltech_autoencoder.evaluate(x_test_caltech, x_test_caltech)
print(f'CalTech109 Binary Cross-Entropy (BCE): {round(caltech_bce, 3)}')

# Calculate M-MSE
def masked_mse(y_true, y_pred, mask):
    return np.mean(((y_true - y_pred) * (1 - mask)) ** 2)

caltech_mask = np.random.choice([0, 1], size=x_test_caltech.shape, p=[0.5, 0.5])
caltech_mse = masked_mse(x_test_caltech, caltech_autoencoder.predict(x_test_caltech), caltech_mask)
print(f'CalTech109 Masked Mean Squared Error (M-MSE): {round(caltech_mse, 3)}')

# Get the latent representations from the encoder
caltech_latent_train = caltech_autoencoder.encoder.predict(x_train_caltech)
caltech_latent_test =  caltech_autoencoder.encoder.predict(x_test_caltech)

# Scale the latent representations
caltech_scaler = StandardScaler()
caltech_latent_train_scaled = caltech_scaler.fit_transform(caltech_latent_train)
caltech_latent_test_scaled =  caltech_scaler.transform(caltech_latent_test)

# Train a logistic regression classifier
classifier = LogisticRegression(max_iter=2000)
classifier.fit(caltech_latent_train_scaled, y_train_caltech)

# Predict on the test set
y_pred = classifier.predict(caltech_latent_test_scaled)

# Calculate classification error
caltech_classification_error = 1 - accuracy_score(y_test_caltech, y_pred)
print(f'Caltech Classification Error: {round(caltech_classification_error, 2):.2%}')

# Caltech likelihood error
log_likelihood = calculate_log_likelihood(caltech_autoencoder, x_test_caltech)
print(f'Caltech Log-Likelihood: {log_likelihood:.2%}')

191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3646
CalTech109 Binary Cross-Entropy (BCE): 0.366
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
CalTech109 Masked Mean Squared Error (M-MSE): 0.057
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Caltech Classification Error: 76.00%
